In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
import time as time
import copy as copy

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [ ]:
XL_PATH = r"radiomicsFeatures.csv"

In [ ]:
feats_df = pd.read_csv(XL_PATH)

feats_df.head()

In [ ]:
feats_df.shape

### Autoencoder Utilities

##### Dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, X):
        self.X = X
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):
        return self.X[i], self.X[i]

In [ ]:
class MixupDataset(torch.utils.data.Dataset):
    
    def __init__(self, X):
        self.X = X
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, i):

        i, j = np.random.randint(len(self.X), size=2)
        
        p = np.random.uniform()
        
        x = p * self.X[i] + (1-p) * self.X[j]
        
        return x, x

##### Architecture

In [ ]:
class FC_Block(nn.Module):
    
    def __init__(self, in_feats, hidden_layers, activation_fn = nn.LeakyReLU()):
        
        super(FC_Block, self).__init__()
        
        layers = []
        
        for out_feats in hidden_layers:
            layers += [nn.Linear(in_feats, out_feats), activation_fn]
            in_feats = out_feats
            
        self.block = nn.Sequential(*layers)
        
    def forward(self, x):
        
        x = self.block(x)
        
        return x
        

In [ ]:
class Autoencoder(nn.Module):
    
    def __init__(self, input_dim, encoder_layers=[100,50,25], latent_dim=5, activation_fn = nn.LeakyReLU()):
        
        super(Autoencoder, self).__init__()
        
        self.encoder_block = FC_Block(input_dim, encoder_layers, activation_fn)
        
        self.embedding_layer = nn.Sequential(*[nn.Linear(encoder_layers[-1], latent_dim), activation_fn])
        
        decoder_layers = list(reversed(encoder_layers))
        self.decoder_block = FC_Block(latent_dim, decoder_layers, activation_fn)
        self.scores = nn.Linear(decoder_layers[-1], input_dim)
    
    def forward(self, x):
        
        x = self.encoder_block(x)
        h = x = self.embedding_layer(x)
        x = self.decoder_block(x)
        x = self.scores(x)
        
        return x, h
            

##### Training/Validation

In [ ]:
class Model:
    
    def __init__(self, net):
        self.net = net
        
    def compile(self, lr, h_lambda, loss_fn, cuda_device_id=0):
        
        self.h_lambda = h_lambda
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr)
        self.loss_fn = loss_fn 
        self.device = torch.device(f"cuda:{cuda_device_id}" if torch.cuda.is_available() else "cpu")
        
        self.net.to(self.device)
        
    def prepare_minibatch(self, mini_batch):
        
        inputs, targets = mini_batch
        
        return inputs.float().to(self.device), targets.float().to(self.device)
        
    def fit(self, dls, num_epochs, verbose=True):
        
        since = time.time()
        
        hist = {'train':{'loss':[]}, 'val':{'loss':[]}}
        
        best_loss = np.inf
        
        for epoch in range(num_epochs):
            
            if verbose:
                
                print('Epoch {}/{}'.format(epoch,num_epochs-1))
                print('-'*10)
                
            for phase in ["train", "val"]:
                
                if phase=="train":
                    self.net.train()
                else:
                    self.net.eval()
                    
                running_loss = 0.0
                
                for mini_batch in dls[phase]:
                    
                    self.optimizer.zero_grad()
                    
                    inputs, targets = self.prepare_minibatch(mini_batch)
                    
                    with torch.set_grad_enabled(phase=="train"):
                        
                        recon_inputs, h = self.net(inputs)
                        
                        loss = self.loss_fn(recon_inputs, targets) + self.h_lambda * h.flatten().abs().sum()
                        
                        if phase=="train":
                            
                            loss.backward()
                            self.optimizer.step()
                            
                        running_loss += loss.item()
                            
                epoch_loss = running_loss/len(dls[phase])
                hist[phase]["loss"].append(epoch_loss)
                
                if verbose:
                    print("{} Loss :{:.4f}".format(phase,epoch_loss))
                    
                if phase == "val":
                    
                    if epoch_loss<best_loss:
                        best_loss = epoch_loss
                        best_model_wts = copy.deepcopy(self.net.state_dict())
                        if verbose:
                            print(f"Checkpoing made at {epoch}")
                        
            if verbose:
                print()
                
            
        time_elapsed = time.time() - since
        
        
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        print('Best val Loss: {:4f}'.format(best_loss)) 

        
        self.net.load_state_dict(best_model_wts)
        
        return self.net.cpu()

In [ ]:
def norm_anomaly_split(X, y):
    
    normal_indeces = np.argwhere(y==0).ravel()
    anomaly_indeces = np.argwhere(y==1).ravel()
    
    X_norm = X[normal_indeces]
    X_anomaly = X[anomaly_indeces]

    return X_norm, X_anomaly
    

In [ ]:
def visualize_using_tsne(X, y, n_components=2):
    
    X_transformed = TSNE(n_components = n_components, random_state=0).fit_transform(X)
    
    plt.scatter(*zip(*X_transformed[y==1]), marker='o', color='r', s=10, label='Anomalous')
    plt.scatter(*zip(*X_transformed[y==0]), marker='o', color='g', s=10, label='Normal')
    plt.legend()
    plt.show()
    
    
    

### Sanity Check - Autoencoder Implementation

In [ ]:
# https://github.com/mvaldenegro/UncertaintyML-course-ESSAI-labs
# https://github.com/mvaldenegro/UncertaintyML-course-ESSAI-labs/blob/main/02_eval_uncertainty_calibration.ipynb
# https://atcold.github.io/NYU-DLSP20/en/week01/01-3/

In [ ]:
feats = [column for column in feats_df.columns if column not in ["id","label"]]
print(len(feats))

In [ ]:
scaler = StandardScaler()
X = feats_df[feats].to_numpy()
y = feats_df["label"].to_numpy()

# X = scaler.fit_transform(X)

X_norm, X_anomaly = norm_anomaly_split(X, y)

X_norm = scaler.fit_transform(X_norm) #this works better than the alternative where you standardize the whole X
X_anomaly = scaler.transform(X_anomaly)

X_norm = torch.from_numpy(X_norm).float()
X_anomaly = torch.from_numpy(X_anomaly).float()

_, input_dim = X.shape

torch.manual_seed(0)
idx = torch.randperm(len(X_norm))

X_train = X_norm[idx[:-len(X_anomaly)]]

X_test_norm = X_norm[idx[-len(X_anomaly):]]
X_test_anomaly = X_anomaly

# X_test = torch.concat([X_test_norm, X_test_anomaly])


In [ ]:
# visualize_using_tsne(X,y)

In [ ]:
num_epochs = 10_000
batch_size = 32
loss_fn = nn.MSELoss()

lr = 1e-3

latent_dim = 5

activation_fn = nn.LeakyReLU()

##### 1. Single Standard Autoencoder

In [ ]:
train_ds = MixupDataset(X_train)
val_ds = Dataset(X_train)
dls = {"train":torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True),"val":torch.utils.data.DataLoader(val_ds, batch_size=batch_size)}

h_lambda = 0.0 #disabling l1 sparsity constraint
encoder_layers = [50, 25, 10] #under-complete hidden layers
standard_ae = Autoencoder(input_dim, encoder_layers=encoder_layers, latent_dim=latent_dim, activation_fn = activation_fn)

model = Model(standard_ae)
model.compile(lr, h_lambda, loss_fn)
_ = model.fit(dls, num_epochs, verbose=True)

In [ ]:
recon_X_test_norm, h = model.net(X_test_norm)
recon_X_test_anomaly, h = model.net(X_test_anomaly)

normal_mse = nn.MSELoss(reduction="none")(recon_X_test_norm, X_test_norm).mean(axis=0).detach().numpy()
anomaly_mse = nn.MSELoss(reduction="none")(recon_X_test_anomaly, X_test_anomaly).mean(axis=0).detach().numpy()

delta = anomaly_mse - normal_mse
rank = len(delta) - (delta.argsort().argsort() + 1) + 1

print(normal_mse.mean(), anomaly_mse.mean())


In [ ]:
plt.hist(normal_mse)
plt.hist(anomaly_mse)

##### 2. Single Sparse Autoencoder

In [ ]:
train_ds = Dataset(X_train)
val_ds = Dataset(X_train)
dls = {"train":torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True),"val":torch.utils.data.DataLoader(val_ds, batch_size=batch_size)}

h_lambda = 1e-2 #with l1 regularization
encoder_layers = [50, 25, 10] #under-complete hidden layers
sparse_ae = Autoencoder(input_dim, encoder_layers=encoder_layers, latent_dim=latent_dim, activation_fn = activation_fn)

model = Model(sparse_ae)
model.compile(lr, h_lambda, loss_fn)
_ = model.fit(dls, num_epochs, verbose=True)

In [ ]:
recon_X_test_norm, h = model.net(X_test_norm)
recon_X_test_anomaly, h = model.net(X_test_anomaly)

normal_mse = nn.MSELoss(reduction="none")(recon_X_test_norm, X_test_norm).mean(axis=0).detach().numpy()
anomaly_mse = nn.MSELoss(reduction="none")(recon_X_test_anomaly, X_test_anomaly).mean(axis=0).detach().numpy()

delta = anomaly_mse - normal_mse
rank = len(delta) - (delta.argsort().argsort() + 1) + 1

print(normal_mse.mean(), anomaly_mse.mean())


In [ ]:
plt.hist(normal_mse)
plt.hist(anomaly_mse)

### Sanity Check - Stability Metrics

##### 3. Ensemble Sparse Autoencoders 

In [ ]:
ensemble_count = 100

num_epochs = 10_000
batch_size = 32
loss_fn = nn.MSELoss()

lr = 1e-3

latent_dim = 5

activation_fn = nn.LeakyReLU()

h_lambda = 1e-2 #with l1 regularization
encoder_layers = [50, 25, 10] #under-complete hidden layers
sparse_ae = Autoencoder(input_dim, encoder_layers=encoder_layers, latent_dim=latent_dim, activation_fn = activation_fn)

results_df = {**{"ae_id":[], "mse_mean":[]}, **{"mse_"+f:[] for f in feats}, **{"label":[]}}

In [ ]:
for ae_id in range(ensemble_count):
    
    print("*"*50)
    print(f"Autoencoder ID: {ae_id+1}")

    scaler = StandardScaler()
    X = feats_df[feats].to_numpy()
    y = feats_df["label"].to_numpy()

    # X = scaler.fit_transform(X)

    X_norm, X_anomaly = norm_anomaly_split(X, y)

    X_norm = scaler.fit_transform(X_norm) #this works better than the alternative where you standardize the whole X
    X_anomaly = scaler.transform(X_anomaly)

    X_norm = torch.from_numpy(X_norm).float()
    X_anomaly = torch.from_numpy(X_anomaly).float()

    _, input_dim = X.shape

    # torch.manual_seed(0) #this is where each ensemble run essentially differ, 
    idx = torch.randperm(len(X_norm))

    X_train = X_norm[idx[:-len(X_anomaly)]]

    X_test_norm = X_norm[idx[-len(X_anomaly):]]
    X_test_anomaly = X_anomaly

    
    train_ds = Dataset(X_train)
    val_ds = Dataset(X_train)
    dls = {"train":torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True),"val":torch.utils.data.DataLoader(val_ds, batch_size=batch_size)}

    model = Model(sparse_ae)
    model.compile(lr, h_lambda, loss_fn)
    _ = model.fit(dls, num_epochs, verbose=False)
    
    recon_X_test_norm, h = model.net(X_test_norm)
    recon_X_test_anomaly, h = model.net(X_test_anomaly)

    normal_mse = nn.MSELoss(reduction="none")(recon_X_test_norm, X_test_norm).mean(axis=0).detach().numpy()
    anomaly_mse = nn.MSELoss(reduction="none")(recon_X_test_anomaly, X_test_anomaly).mean(axis=0).detach().numpy()
    
    
    results_df["ae_id"].append(ae_id+1)
    results_df["mse_mean"].append(normal_mse.mean())
    for f, f_normal_mse in zip(feats, normal_mse):
        results_df["mse_"+f].append(f_normal_mse)
    results_df["label"].append(0)
        
    results_df["ae_id"].append(ae_id+1)
    results_df["mse_mean"].append(anomaly_mse.mean())
    for f, f_anomaly_mse in zip(feats, anomaly_mse):
        results_df["mse_"+f].append(f_anomaly_mse)
    results_df["label"].append(1)
        
    print("normal_mse=", normal_mse.mean(), "anomaly_mse=", anomaly_mse.mean())
    
results_df = pd.DataFrame(results_df)

In [ ]:
results_df.to_csv("results_df.csv", index=False)